# Final Project

Joy Wang, Stephanie Wang, Lucinda Zhou 

CMSC320

# Data Curation and Parsing

In [33]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

In [34]:
# read in data and store as a dataframe
df = pd.read_csv('https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/covid_tracking/latest/covid_tracking.csv')

# remove deprecated columns based on data documentation
df = df.drop(df.columns[df.columns.str.contains('increase')], axis=1)
df = df.drop(labels=['date_checked','hospitalized','pos_neg','total'], axis=1)

df.head()

,date,state,positive,negative,pending,hospitalized_currently,hospitalized_cumulative,in_icu_currently,in_icu_cumulative,on_ventilator_currently,...,data_quality_grade,last_update_et,hash,death,total_test_results,fips,fips_code,iso_subdivision,load_time,iso_country
0,2021-03-07,AK,56886.0,NaN,NaN,33.0,1293.0,NaN,NaN,2.0,...,NaN,2021-03-05 03:59:00,dc4bccd4bb885349d7e94d6fed058e285d4be164,305.0,1731628.0,2,2,US-AK,2022-01-11 00:04:58,US
1,2021-03-07,AL,499819.0,1931711.0,NaN,494.0,45976.0,NaN,2676.0,NaN,...,NaN,2021-03-07 11:00:00,997207b430824ea40b8eb8506c19a93e07bc972e,10148.0,2323788.0,1,1,US-AL,2022-01-11 00:04:58,US
2,2021-03-07,AR,324818.0,2480716.0,NaN,335.0,14926.0,141.0,NaN,65.0,...,NaN,2021-03-07 00:00:00,50921aeefba3e30d31623aa495b47fb2ecc72fae,5319.0,2736442.0,5,5,US-AR,2022-01-11 00:04:58,US
3,2021-03-07,AS,0.0,2140.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2020-12-01 00:00:00,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0.0,2140.0,60,60,US-AS,2022-01-11 00:04:58,US
4,2021-03-07,AZ,826454.0,3073010.0,NaN,963.0,57907.0,273.0,NaN,143.0,...,NaN,2021-03-07 00:00:00,0437a7a96f4471666f775e63e86923eb5cbd8cdf,16328.0,7908105.0,4,4,US-AZ,2022-01-11 00:04:58,US


For our final project, we decided to analyze the dataset from the COVID Tracking project, which is one of the open datasets from Microsoft. We utilized the CSV version of the dataset, and used pandas read_csv() to read and parse the csv file into a pandas dataframe. 

While reading Microsoft's documentation for this dataset, we noticed that some columns were described as "Deprecated", so we did some preliminary data cleaning by dropping those columns from the datafram using the drop() function.

# Joy's Work in Progress :))

In [35]:
# lin reg models and hypothesis testing
# consider: how am i dealing with nan? by removing rows with nan values in the columns im modeling

# dropping unnecessary columns for my linear regression models
lrdata = df.drop(df.columns[df.columns.str.contains('cumulative|iso')], axis=1)
lrdata = lrdata.drop(labels=['pending','data_quality_grade','last_update_et','hash','fips','fips_code','load_time'], axis=1)
lrdata

,date,state,positive,negative,hospitalized_currently,in_icu_currently,on_ventilator_currently,recovered,death,total_test_results
0,2021-03-07,AK,56886.0,NaN,33.0,NaN,2.0,NaN,305.0,1731628.0
1,2021-03-07,AL,499819.0,1931711.0,494.0,NaN,NaN,295690.0,10148.0,2323788.0
2,2021-03-07,AR,324818.0,2480716.0,335.0,141.0,65.0,315517.0,5319.0,2736442.0
3,2021-03-07,AS,0.0,2140.0,NaN,NaN,NaN,NaN,0.0,2140.0
4,2021-03-07,AZ,826454.0,3073010.0,963.0,273.0,143.0,NaN,16328.0,7908105.0
...,...,...,...,...,...,...,...,...,...,...
22256,2020-01-17,WA,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22257,2020-01-16,WA,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22258,2020-01-15,WA,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22259,2020-01-14,WA,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# getting a list with all unique states
states = lrdata['state'].unique()

# create new dataframe with averages for each state
state_avgs = pd.DataFrame(columns=['state', 'positive_avg','hospitalized_currently_avg','recovered_avg','death_avg'])
state_avgs['state'] = states
state_avgs.head()

,state,positive_avg,hospitalized_currently_avg,recovered_avg,death_avg
0,AK,NaN,NaN,NaN,NaN
1,AL,NaN,NaN,NaN,NaN
2,AR,NaN,NaN,NaN,NaN
3,AS,NaN,NaN,NaN,NaN
4,AZ,NaN,NaN,NaN,NaN


In [37]:
# calculating averages for each of these columns for each state
for column in ['positive','hospitalized_currently','recovered','death']:
    # dropping rows with NaN values in the current column we are operating on
    lrdata_col = lrdata.dropna(subset=[column])
    # there are 56 states, and we will iterate through them
    for i in range(56):
        # create a dataframe that only contains rows for the current state
        state = states[i]
        lrdata_state = lrdata_col[lrdata_col['state'] == states[i]]
        # calculate the average for this state and this column
        average = lrdata_state[column].mean()
        # insert the average value into the dataframe where we are storing averages for each state
        state_avgs.at[i, (column+'_avg')] = average
state_avgs.head()

# FINISH CREATING THE AVGS TABLE!!!

# create plot and lin reg model for 3 relationships with positive numbers

,state,positive_avg,hospitalized_currently_avg,recovered_avg,death_avg
0,AK,17298.2,54.9506,2153.11,85.7929
1,AL,169569,1160.32,109246,2900.18
2,AR,102514,535.738,97263.8,1713.85
3,AS,0,NaN,NaN,0
4,AZ,258711,1947.81,NaN,5328.98


# Stephanie's WIP

We can't do k means analysis on non numerical variables, so we can drop those. 

In [4]:
mldata = df.copy()
mldata = mldata.drop(
    labels=['data_quality_grade', 'last_update_et', 'hash', 'load_time', 'iso_subdivision', 'iso_country'], 
    axis=1
)
mldata.head()

,date,state,positive,negative,pending,hospitalized_currently,hospitalized_cumulative,in_icu_currently,in_icu_cumulative,on_ventilator_currently,on_ventilator_cumulative,recovered,death,total_test_results,fips,fips_code
0,2021-03-07,AK,56886.0,NaN,NaN,33.0,1293.0,NaN,NaN,2.0,NaN,NaN,305.0,1731628.0,2,2
1,2021-03-07,AL,499819.0,1931711.0,NaN,494.0,45976.0,NaN,2676.0,NaN,1515.0,295690.0,10148.0,2323788.0,1,1
2,2021-03-07,AR,324818.0,2480716.0,NaN,335.0,14926.0,141.0,NaN,65.0,1533.0,315517.0,5319.0,2736442.0,5,5
3,2021-03-07,AS,0.0,2140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2140.0,60,60
4,2021-03-07,AZ,826454.0,3073010.0,NaN,963.0,57907.0,273.0,NaN,143.0,NaN,NaN,16328.0,7908105.0,4,4


In [5]:
mldata = mldata.drop(
    labels=['date', 'hospitalized_currently', 'in_icu_currently', 'on_ventilator_currently', 'fips', 'fips_code'], 
    axis=1
)
mldata.head()

,state,positive,negative,pending,hospitalized_cumulative,in_icu_cumulative,on_ventilator_cumulative,recovered,death,total_test_results
0,AK,56886.0,NaN,NaN,1293.0,NaN,NaN,NaN,305.0,1731628.0
1,AL,499819.0,1931711.0,NaN,45976.0,2676.0,1515.0,295690.0,10148.0,2323788.0
2,AR,324818.0,2480716.0,NaN,14926.0,NaN,1533.0,315517.0,5319.0,2736442.0
3,AS,0.0,2140.0,NaN,NaN,NaN,NaN,NaN,0.0,2140.0
4,AZ,826454.0,3073010.0,NaN,57907.0,NaN,NaN,NaN,16328.0,7908105.0


Getting cumulative values for each column 

In [6]:
bystate = mldata.groupby('state').max()
bystate

,positive,negative,pending,hospitalized_cumulative,in_icu_cumulative,on_ventilator_cumulative,recovered,death,total_test_results
state,,,,,,,,,
AK,56886.0,NaN,14.0,1293.0,NaN,NaN,7165.0,305.0,1731628.0
AL,499819.0,1931711.0,46.0,45976.0,2676.0,1515.0,295690.0,10149.0,2323788.0
AR,324818.0,2480716.0,203.0,14926.0,43.0,1533.0,315517.0,5417.0,2736442.0
AS,0.0,2140.0,17.0,NaN,NaN,NaN,NaN,0.0,2140.0
AZ,826454.0,3073010.0,130.0,57907.0,NaN,NaN,NaN,16328.0,7908105.0
CA,3501394.0,266839.0,15000.0,NaN,NaN,NaN,NaN,32291.0,49646014.0
CO,436602.0,2199458.0,69.0,23904.0,NaN,NaN,NaN,5989.0,6415123.0
CT,285330.0,NaN,8.0,13793.0,NaN,NaN,9800.0,7704.0,6520366.0
DC,41419.0,NaN,20.0,NaN,NaN,NaN,29570.0,1030.0,1261363.0


In [7]:
bystate = bystate.dropna(axis=1)
bystate

,positive,death,total_test_results
state,,,
AK,56886.0,305.0,1731628.0
AL,499819.0,10149.0,2323788.0
AR,324818.0,5417.0,2736442.0
AS,0.0,0.0,2140.0
AZ,826454.0,16328.0,7908105.0
CA,3501394.0,32291.0,49646014.0
CO,436602.0,5989.0,6415123.0
CT,285330.0,7704.0,6520366.0
DC,41419.0,1030.0,1261363.0


In [8]:
km = KMeans(
    n_clusters=3, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
# y_km = km.fit_predict(X)